### Добро пожаловать на кейс МЧС "Прогнозирование опасностей и рисков Пермского края"!
*** 
В представленном архиве вы можете увидеть файлы и папки следующего содержания.

1. Файл **Данные по метеостанциям.csv** - набор погодных замеров различных метеостанций Пермского края.
2. Файл **Данные по метеостанциям. Соответствие МО.csv** - соответствие метеостанций и муниципальных округов.
3. Файл **Данные по метеостанциям. Описания столбцов.txt** - описания столбцов файла **Данные по метеостанциям.csv**
4. **ДТП.csv** - список ДТП различных категорий 2013-2022 гг.
5. **Износ сетей по МО ПК.csv**
6. **ОЯ и НЯ.csv** - список погодных предсказаний и их подтверждений различных категорий 2013-2022 гг. Можно считать, что информация о прогнозе известна за 10 дней до непосредственной даты. 
7. **Пожары.csv** - список пожаров различных категорий 2013-2022 гг.
8. **Происшествия.csv** - список происшествий различных категорий 2013-2022 гг.
9. **Температурная статистика.csv** - температурная статистика по Пермскому краю по дням различных годов.

***

##### Вашей задачей будет разработка модели и интерфейса для прогнозирования рисков происшествий по Пермскому краю.

Модель представляет собой прогностический инструмент, который предсказывает вероятность происшествия, для формирования целевой переменной обязательно использовать файл **Происшествия.csv**. Прогноз осуществляется на основе информации, имеющейся на заданный день (историческая информация о погодных условиях/прогнозы/произвольные открытые данные, которые вы найдете и обоснуете/информация о происшествиях в прошлые моменты времени по файлам "пожары", "дтп", "НЯ и ОЯ"), на 10 дней вперед, прогноз необходим для каждого дня горизонта в виде вероятности заданного типа. 

Базово необходимо осуществлять прогнозы по следующим типам:
1. Аварии на транспорте,
2. Аварии с выбросом опасных/токсичных веществ,
3. Опасные природные явления,
4. ЖКХ,
5. Взрывы/пожары/разрушения,
6. Прочие опасности.

Дополнительными баллами оценивается построение модели по расширенному списку рисков:
1. Аварии на железнодорожном транспорте,
2. Аварии на автомобильном транспорте,
3. Аварии на водном транспорте,
4. Аварии на воздушном транспорте,
5. Взрывы (в том числе с последующим горением) и/или разрушения (обрушения) в зданиях и сооружениях,
6. Аварии на системах жизнеобеспечения,
7. Аварии с выбросом, сбросом опасных химических веществ,
8. Аварии с розливом (выбросом) нефти, нефтепродуктов,
9. Аварии с выбросом (проливом, просыпом) патогенных для человека микроорганизмов,
10. Гидродинамические аварии,
11. Опасные геологические явления, 
12. Опасные метеорологические явления, 
13. Опасные гидрологические явления,
14. Опасные явления в лесах,
15. Космические опасности,
16. Биологическая опасность.

Автоматическая проверка в данном кейсе отсутствует. Для решения задачи необходимо формализовать метрику качества (например, усредненный ROC AUC по типам рисков), особосновать ее выбор и осуществить контроль за ее стабильностью с помощью валидации и тестирования. 

# ЖЕЛАЕМ УДАЧИ!

P.S. Не забудьте посетить экспертные сессии и не стесняйтесь задавать вопросы)

# installing modules

In [1]:
#!pip install -q numpy
#!pip install -q pandas
#!pip install -q scikit-learn
#!pip install -q joblib
#!pip install -q tabpfn

# Import modules

In [2]:
import joblib
import numpy as np
import pandas as pd

from sklearn.metrics import precision_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, recall_score

from tabpfn import TabPFNClassifier

# Data handling

## Data loading

In [3]:
df_car_acidents = pd.read_csv('train_data/Данные/ДТП.csv')
df_meteo = pd.read_csv('train_data/Данные/Данные по метеостанциям.csv')
df_network_wear = pd.read_csv('train_data/Данные/Износ сетей по МО ПК.csv')
df_oy_ny = pd.read_csv('train_data/Данные/ОЯ и НЯ.csv')
df_fires = pd.read_csv('train_data/Данные/Пожары.csv')
df_accidents = pd.read_csv('train_data/Данные/Происшествия.csv')
df_temp_stat = pd.read_csv('train_data/Данные/Температурная статистика.csv')
df_meteo_mo = pd.read_csv('train_data/Данные/Данные по метеостанциям. Соответствие МО.csv')

/tmp/ipykernel_102810/3043878564.py:2: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df_meteo = pd.read_csv('train_data/Данные/Данные по метеостанциям.csv')


In [4]:
df_car_acidents.head()

,Район,Тип выезда,Дата и время пожара,Причина
0,Пермский ГО,ДТП,2013-01-01 01:55:00,ДТП/Столкновение
1,ГО город Березники,ДТП,2013-01-01 04:20:00,ДТП/Столкновение наез на препятствие (столб)
2,Ильинский ГО,ДТП,2013-01-01 05:30:00,ДТП/Столкновение
3,Осинский ГО,ДТП,2013-01-01 06:19:00,ДТП/Столкновение опрокидывание в кювет
4,Куединский МО,ДТП,2013-01-01 08:00:00,ДТП/Столкновение


In [5]:
df_meteo.head()

,Местное время,T,Po,P,Pa,U,DD,Ff,ff10,ff3,...,Ch,VV,Td,RRR,tR,E,Tg,E',sss,meteostation
0,31.12.2022 23:00,-18.6,761.0,770.9,NaN,73.0,"Ветер, дующий с востока",1.0,NaN,NaN,...,NaN,NaN,-22.2,NaN,NaN,NaN,NaN,NaN,NaN,Оса
1,31.12.2022 20:00,-18.0,761.5,771.4,NaN,72.0,"Ветер, дующий с востока",1.0,NaN,NaN,...,NaN,4,-21.8,Следы осадков,12.0,NaN,NaN,NaN,NaN,Оса
2,31.12.2022 17:00,-17.0,761.9,771.8,NaN,73.0,"Ветер, дующий с востока",1.0,NaN,NaN,...,"Перистых, перисто-кучевых или перисто-слоистых...",10,-20.8,NaN,NaN,NaN,NaN,NaN,NaN,Оса
3,31.12.2022 14:00,-16.2,762.4,772.3,NaN,68.0,"Ветер, дующий с востоко-северо-востока",2.0,NaN,NaN,...,"Перисто-кучевые одни или перисто-кучевые, сопр...",10,-20.8,NaN,NaN,NaN,NaN,NaN,NaN,Оса
4,31.12.2022 11:00,-17.4,762.5,772.4,NaN,72.0,"Ветер, дующий с востоко-северо-востока",2.0,NaN,NaN,...,"Перистые нитевидные, иногда когтевидные, не ра...",20,-21.3,NaN,NaN,NaN,NaN,NaN,NaN,Оса


In [6]:
df_network_wear.head()

,Наименование муниципального образования,Сетей теплоснабжения,Сетей холодного водоснабжения,Сетей горячего водоснабжения,Сетей электроснабжения,Сети газоснабжения,Станций водоподготовки,Водонасосных объектов,Объектов водозабора,Сетей водоотведения,Очистных сооружений канализации,Канализационных насосных станций,Котельных,Тепловых пунктов
0,Чердынский городской округ,70.00,80.00,NaN,70.0,NaN,NaN,80.00,65.00,95.0,100.00,100.00,70.00,NaN
1,Оханский городской округ,NaN,80.00,0.0,NaN,53.11,NaN,NaN,NaN,90.0,80.00,NaN,NaN,NaN
2,Кишертский муниципальный округ,57.00,75.00,0.0,0.0,35.00,65.0,80.00,75.00,95.8,99.80,99.80,87.00,NaN
3,Сивинский муниципальный округ,81.05,91.88,0.0,0.0,0.00,0.0,0.00,100.00,100.0,78.50,51.50,55.71,0
4,Соликамский городской округ,80.00,73.00,80.0,55.0,10.00,66.0,66.05,36.93,73.0,75.62,55.25,80.00,80


In [7]:
df_oy_ny.head()

,"Дата, время начала",Районы,подтвержденные (прогноз оправдался),"НЯ, ОЯ, КНЯ",Тип явления,Критерий,Характеристика,Последствия
0,2014-12-06 21:00:00,ВСЕ муниципальные образования,+,НЯ,Метеорологическое,Сильный мороз,06.12.14 по северо-востоку края сохранится сил...,NaN
1,2014-12-21 00:00:00,ВСЕ муниципальные образования,+,НЯ,Метеорологическое,"Очень сильный снег , Сильная метель","ночью 21.12.2014, с сохранением днем, ожидаетс...",NaN
2,2015-01-01 00:00:00,Местами по краю,NaN,НЯ,Метеорологическое,Сильный мороз,1 и 2 января 2015г местами по Пермскому краю о...,NaN
3,2015-01-03 06:00:00,Местами по краю,NaN,НЯ,Метеорологическое,"Очень сильный ветер , Очень сильный снег , Си...","Днем 03.01 сильный снег, метель, порывы ветра ...",NaN
4,2015-01-07 00:00:00,Местами по краю,NaN,ОЯ,Метеорологическое,Сильный мороз,с 07.01.2015 по 10.01.2015 ожидается сильный м...,NaN


In [8]:
df_fires.head()

,Район,Тип выезда,Дата и время пожара,Причина
0,Пермский ГО,Пожар,2013-01-21 00:00:00,Умышл. действия по уничтож. (поврежд.) имущест...
1,Добрянский ГО,Пожар,2013-01-22 10:33:00,Нарушение правил тех. эксплуатации эл.оборудов...
2,Ильинский ГО,Пожар,2013-01-23 21:30:00,Умышл. действия по уничтож. (поврежд.) имущест...
3,Кунгурский МО,Пожар,2013-01-26 13:20:00,Умышл. действия по уничтож. (поврежд.) имущест...
4,Ильинский ГО,Пожар,2013-01-27 13:00:00,НППБ при эксплуатации бытовых эл.приборов


In [9]:
df_accidents.head()

,Район,Тип,Дата и время пожара
0,ГО город Березники,Пожар с гибелью 2-х и более человек,2012-12-18 20:39:00
1,Пермский МО,Особо опасные болезни сельскохозяйственных жив...,2013-01-01 00:00:00
2,Юрлинский МО,Особо опасные болезни сельскохозяйственных жив...,2013-01-01 00:00:00
3,Карагайский МО,Особо опасные болезни сельскохозяйственных жив...,2013-01-01 00:00:00
4,Добрянский ГО,Пожар с гибелью 2-х и более человек,2013-01-01 07:50:00


In [10]:
df_temp_stat.head()

,year,month,day,t
0,среднемноголетняя температура,январь,1,-9.857143
1,2009,январь,1,-8.000000
2,2010,январь,1,-13.000000
3,2011,январь,1,-5.000000
4,2012,январь,1,-5.000000


In [11]:
df_meteo_mo.head()

,Муниципальное образование,Метеорологическая станция
0,Александровский МО,г. Березники
1,Бардымский МО,г. Оса
2,Березовский МО,г. Кунгур
3,Большесосновский МО,с. Большая Соснова
4,Верещагинский ГО,г. Верещагино


## Common data transformations

### df_temp_stat

In [12]:
df_temp_stat['year'].unique()

array(['среднемноголетняя температура', '2009', '2010', '2011', '2012',
       '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020',
       '2021', '2022', '2023'], dtype=object)

In [13]:
df_temp_stat = df_temp_stat[df_temp_stat['year'] != "среднемноголетняя температура"]

In [14]:
df_temp_stat['year'].unique()

array(['2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016',
       '2017', '2018', '2019', '2020', '2021', '2022', '2023'],
      dtype=object)

In [15]:
df_temp_stat['month'].unique()

array(['январь', 'ферваль', 'март', 'апрель', 'май', 'июнь', 'июль',
       'август', 'сентябрь', 'октябрь', 'ноябрь', 'декабрь'], dtype=object)

In [16]:
value_mapping = {'январь': 'january', 
                 'ферваль': 'february',
                 'март': 'march', 
                 'апрель': 'april',
                 'май': 'may', 
                 'июнь': 'june',
                 'июль': 'july', 
                 'август': 'august',
                 'сентябрь': 'september', 
                 'октябрь': 'october',
                 'ноябрь': 'november', 
                 'декабрь': 'december',}

df_temp_stat['month'] = df_temp_stat['month'].replace(value_mapping)

In [17]:
df_temp_stat['month'].unique()

array(['january', 'february', 'march', 'april', 'may', 'june', 'july',
       'august', 'september', 'october', 'november', 'december'],
      dtype=object)

В данных в феврале всегда 29 дней, даже не в високосный год 🤔🤔🤔

In [18]:
for month in df_temp_stat['month'].unique():
    selected_elements = df_temp_stat[(df_temp_stat['month'] == month) & (df_temp_stat['year'] == '2009')]
    print(f"Month: {month} | n_days: {selected_elements.shape[0]}")

Month: january | n_days: 31
Month: february | n_days: 29
Month: march | n_days: 31
Month: april | n_days: 30
Month: may | n_days: 31
Month: june | n_days: 30
Month: july | n_days: 31
Month: august | n_days: 31
Month: september | n_days: 30
Month: october | n_days: 31
Month: november | n_days: 30
Month: december | n_days: 31


In [19]:
df_temp_stat = df_temp_stat[(df_temp_stat['month'] != 'february') & (df_temp_stat['day'] != 29)]
df_temp_stat['Дата'] = df_temp_stat['year'].astype(str) + '-' + df_temp_stat['month'] + '-' + df_temp_stat['day'].astype(str)

df_temp_stat['Дата'] = pd.to_datetime(df_temp_stat['Дата'], format='%Y-%B-%d',)
df_temp_stat['Дата'] = df_temp_stat['Дата'].dt.date

df_temp_stat = df_temp_stat[['Дата', 't']]
df_temp_stat

,Дата,t
1,2009-01-01,-8.0
2,2010-01-01,-13.0
3,2011-01-01,-5.0
4,2012-01-01,-5.0
5,2013-01-01,-18.0
...,...,...
5821,2019-12-31,-11.0
5822,2020-12-31,-15.0
5823,2021-12-31,-14.0
5824,2022-12-31,-17.0


In [20]:
# df_temp_stat.to_csv('temperature_stats.csv')

## Data for road accidents
Datasets used for trainning, validating and predicting road accidents: 
* df_car_acidents,
* df_accidents,
* df_meteo, 
* df_oy_ny,
* df_temp_stat

### Car acidents

Знаменитый славянский наезд на пешехода в 1120г 🤔🤔🤔

In [21]:
df_car_acidents.iloc[22193]

Район                           Пермский ГО   
Тип выезда                                ДТП 
Дата и время  пожара         21.01.1120 17:50 
Причина                  ДТП/Наезд на пешехода
Name: 22193, dtype: object

In [22]:
row_to_exclude = 22193

# Extract rows before and after the row to be excluded
df_car_acidents = pd.concat([df_car_acidents.iloc[:row_to_exclude], df_car_acidents.iloc[row_to_exclude + 1:]])

In [23]:
df_car_acidents.shape

(30698, 4)

In [24]:
df_car_acidents['Дата и время  пожара '] = pd.to_datetime(df_car_acidents['Дата и время  пожара '])
df_car_acidents['Дата'] = df_car_acidents['Дата и время  пожара '].dt.date
df_car_acidents = df_car_acidents[['Дата', 'Причина']]

In [25]:
df_car_acidents.head()

,Дата,Причина
0,2013-01-01,ДТП/Столкновение
1,2013-01-01,ДТП/Столкновение наез на препятствие (столб)
2,2013-01-01,ДТП/Столкновение
3,2013-01-01,ДТП/Столкновение опрокидывание в кювет
4,2013-01-01,ДТП/Столкновение


### Acidents

In [26]:
df_accidents.head()

,Район,Тип,Дата и время пожара
0,ГО город Березники,Пожар с гибелью 2-х и более человек,2012-12-18 20:39:00
1,Пермский МО,Особо опасные болезни сельскохозяйственных жив...,2013-01-01 00:00:00
2,Юрлинский МО,Особо опасные болезни сельскохозяйственных жив...,2013-01-01 00:00:00
3,Карагайский МО,Особо опасные болезни сельскохозяйственных жив...,2013-01-01 00:00:00
4,Добрянский ГО,Пожар с гибелью 2-х и более человек,2013-01-01 07:50:00


In [27]:
df_accidents['Дата'] = pd.to_datetime(df_accidents['Дата и время  пожара '])
df_accidents['Дата'] = df_accidents['Дата'].dt.date
df_accidents = df_accidents[['Дата', 'Тип']]
df_accidents.head()

,Дата,Тип
0,2012-12-18,Пожар с гибелью 2-х и более человек
1,2013-01-01,Особо опасные болезни сельскохозяйственных жив...
2,2013-01-01,Особо опасные болезни сельскохозяйственных жив...
3,2013-01-01,Особо опасные болезни сельскохозяйственных жив...
4,2013-01-01,Пожар с гибелью 2-х и более человек


### Oy Ny

In [28]:
df_oy_ny.head(1)

,"Дата, время начала",Районы,подтвержденные (прогноз оправдался),"НЯ, ОЯ, КНЯ",Тип явления,Критерий,Характеристика,Последствия
0,2014-12-06 21:00:00,ВСЕ муниципальные образования,+,НЯ,Метеорологическое,Сильный мороз,06.12.14 по северо-востоку края сохранится сил...,NaN


In [29]:
df_oy_ny['Дата'] = pd.to_datetime(df_oy_ny['Дата, время начала'])
df_oy_ny['Дата'] = df_oy_ny['Дата'].dt.date

df_oy_ny = df_oy_ny[['Дата', 'Критерий', 'Тип явления',]]

In [30]:
df_oy_ny.head(1)

,Дата,Критерий,Тип явления
0,2014-12-06,Сильный мороз,Метеорологическое


### Meteo

In [31]:
df_meteo.head(1)

,Местное время,T,Po,P,Pa,U,DD,Ff,ff10,ff3,...,Ch,VV,Td,RRR,tR,E,Tg,E',sss,meteostation
0,31.12.2022 23:00,-18.6,761.0,770.9,NaN,73.0,"Ветер, дующий с востока",1.0,NaN,NaN,...,NaN,NaN,-22.2,NaN,NaN,NaN,NaN,NaN,NaN,Оса


In [32]:
df_meteo.shape

(569516, 30)

In [33]:
df_meteo.columns

Index(['Местное время', 'T', 'Po', 'P', 'Pa', 'U', 'DD', 'Ff', 'ff10', 'ff3',
       'N', 'WW', 'W1', 'W2', 'Tn', 'Tx', 'Cl', 'Nh', 'H', 'Cm', 'Ch', 'VV',
       'Td', 'RRR', 'tR', 'E', 'Tg', 'E'', 'sss', 'meteostation'],
      dtype='object')

In [34]:
df_meteo['Дата'] = pd.to_datetime(df_meteo['Местное время'])
df_meteo['Дата'] = df_meteo['Дата'].dt.date
feature_columns = ['T', 'Po', 'P', 'Pa', 'U', 'DD', 'Ff', 'ff10', 'ff3',
       'N', 'WW', 'W1', 'W2', 'Tn', 'Tx', 'Cl', 'Nh', 'H', 'Cm', 'Ch', 'VV',
       'Td', 'RRR', 'tR', 'E', 'Tg', "E'", 'sss',]

df_meteo = df_meteo[['Дата', *feature_columns]]

In [35]:
df_meteo.head()

,Дата,T,Po,P,Pa,U,DD,Ff,ff10,ff3,...,Cm,Ch,VV,Td,RRR,tR,E,Tg,E',sss
0,2022-12-31,-18.6,761.0,770.9,NaN,73.0,"Ветер, дующий с востока",1.0,NaN,NaN,...,NaN,NaN,NaN,-22.2,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-12-31,-18.0,761.5,771.4,NaN,72.0,"Ветер, дующий с востока",1.0,NaN,NaN,...,NaN,NaN,4,-21.8,Следы осадков,12.0,NaN,NaN,NaN,NaN
2,2022-12-31,-17.0,761.9,771.8,NaN,73.0,"Ветер, дующий с востока",1.0,NaN,NaN,...,Клочья (часто чечевицеобразные) высококучевых ...,"Перистых, перисто-кучевых или перисто-слоистых...",10,-20.8,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-12-31,-16.2,762.4,772.3,NaN,68.0,"Ветер, дующий с востоко-северо-востока",2.0,NaN,NaN,...,"Высококучевые просвечивающие, расположенные на...","Перисто-кучевые одни или перисто-кучевые, сопр...",10,-20.8,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-12-31,-17.4,762.5,772.4,NaN,72.0,"Ветер, дующий с востоко-северо-востока",2.0,NaN,NaN,...,"Высококучевые, просвечивающие или плотные в дв...","Перистые нитевидные, иногда когтевидные, не ра...",20,-21.3,NaN,NaN,NaN,NaN,NaN,NaN


### Выделение осн. категорий событий 

In [36]:
accident_types = df_accidents['Тип'].unique()
accident_types = [item for item in accident_types if "транс" in item]

for item in df_car_acidents['Причина'].unique():
    if not isinstance(item, float):
        accident_types.append(item)

replacements = {
    "стол": "ДТП/Столкновение",
    "наез": "ДТП/Наезд",
    "опрок": "ДТП/Опрокидывание",
    "опрак": "ДТП/Опрокидывание",
    "упал": "ДТП/Опрокидывание",
    "опрк": "ДТП/Опрокидывание",
    "съе": "ДТП/Съезд",
    "кьюве": "ДТП/Съезд",
    "нарушен": "Нарушение ПДД",
    "неисправн": "Неисправность транспортного средства",
    "падени": "ДТП/Падение пассажира",
    "выпа": "ДТП/Падение пассажира",
    "возг": "Возгорание транспортого средства",
    "загор": "Возгорание транспортого средства",
    "кз": "Неисправность транспортного средства",
    "к/з": "Неисправность транспортного средства",
    "справ": "Не справился с управлением"

}
general_accident_types = []

for ind, item in enumerate(accident_types):
    # print(type(item), ind)
    for key in replacements:
        # print(item)
        if key in item.lower():
            accident_types[ind] = replacements[key]
            if replacements[key] not in general_accident_types:
                general_accident_types.append(replacements[key])
            break
            
set(accident_types)

{' невнимательность водителя',
 '(провал в яму)',
 'Аварии (катастрофы) на автодорогах (на транспорте, перевозящем опасные грузы) ',
 'Аварийные ситуации на ЖД транспорте ',
 'Взрывы',
 'Возгорание транспортого средства',
 'ДТП',
 'ДТП/Наезд',
 'ДТП/Опрокидывание',
 'ДТП/Падение пассажира',
 'ДТП/Столкновение',
 'ДТП/Съезд',
 'Нарушение ПДД',
 'Не справился с управлением',
 'Неисправность транспортного средства',
 'Неустановленные причины',
 'Пожары на транспортных средствах (ж/д, водный, автомобильный, воздушный) ',
 'Попадание масла на турбонагнетатель',
 'Прочие причины, связ. с неосторожным обращением с огнем виновное лицо устанавливается',
 'Розлив топлива',
 'Умышл. действия по уничтож. (поврежд.) имущества, нанесению вреда здоровью чел. (поджог)',
 'а/м и велосипедист',
 'газовый баллон пропускал газ в салон',
 'гололёд',
 'иной вид ДТП',
 'короткое замыкание эл.проводки в моторном отсеке',
 'обрыв заправочной колонки на АЗС',
 'отбрасывание предмета с буксирующей петли в прохож

Путем обобщения выделены основные классы *Аварий на транспорте*

In [37]:
general_accident_types

['ДТП/Столкновение',
 'ДТП/Наезд',
 'ДТП/Съезд',
 'ДТП/Опрокидывание',
 'Не справился с управлением',
 'Неисправность транспортного средства',
 'Нарушение ПДД',
 'ДТП/Падение пассажира',
 'Возгорание транспортого средства']

### Выбор общих дат для обучения и валидации


In [38]:
# df_car_acidents, df_accidents, df_meteo, df_oy_ny,df_temp_stat
final_dates = pd.merge(df_car_acidents['Дата'], df_accidents['Дата'], on='Дата', how='inner')
final_dates = pd.merge(final_dates, df_oy_ny['Дата'], on='Дата', how='inner')
final_dates = pd.merge(final_dates, df_temp_stat['Дата'], on='Дата', how='inner')
final_dates = pd.merge(final_dates, df_temp_stat['Дата'], on='Дата', how='inner')

final_dates = final_dates.sort_values(by='Дата', ascending=True)

In [39]:
final_dates.shape

(5665, 1)

In [40]:
final_dates =  final_dates['Дата'].unique()

In [41]:
final_dates.shape

(238,)

### Creating X, y dataframes 

In [42]:
window = 10

X

In [43]:
df_final_dates = pd.DataFrame()
df_final_dates['Дата'] = final_dates
X = pd.merge(df_meteo, df_final_dates, on='Дата', how='inner')
X = pd.merge(X, df_temp_stat, on='Дата', how='inner')

In [44]:
# group by date and calculate mean of number column
X = X.groupby('Дата').mean().reset_index()
X = X[[*X.columns[1:]]]

/tmp/ipykernel_102810/3952792052.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  X = X.groupby('Дата').mean().reset_index()


In [45]:
mean_values = X.mean()
X.fillna(mean_values, inplace=True)

In [46]:
X.shape

(238, 14)

In [47]:
X.head()

,T,Po,P,Pa,U,Ff,ff10,ff3,Tn,Tx,Td,tR,Tg,t
0,12.545890,743.395205,761.344521,0.179861,78.833333,2.349315,12.000000,10.777778,9.300000,16.658824,8.672917,12.0,9.526316,-11.0
1,-1.057534,730.862759,749.433103,-0.026761,94.910959,3.842466,10.875000,11.157895,-3.210000,-0.294444,-1.786301,12.0,7.799509,0.0
2,-27.423288,738.335417,759.324306,0.010000,76.219178,0.904110,10.817174,11.160562,-30.036364,-24.677778,-30.335616,12.0,7.799509,-20.0
3,-6.238621,753.678621,773.291724,-0.264085,75.372414,2.124138,10.817174,11.160562,-10.410000,-0.472222,-10.277397,12.0,7.799509,-16.0
4,15.377397,738.968310,756.580986,-0.248905,90.493151,2.000000,10.817174,10.500000,14.072727,17.272222,13.772603,12.0,15.105263,-10.0


In [48]:
X.to_csv("X_DTP.csv")

y

In [49]:
column_names = ['Дата', *general_accident_types]

# Create a DataFrame
y = pd.DataFrame(np.zeros((len(final_dates), len(column_names)), dtype=int), columns=column_names)

# Set the values in the first column
y['Дата'] = final_dates

In [50]:
y.shape

(238, 10)

In [51]:
y.head(3)

,Дата,ДТП/Столкновение,ДТП/Наезд,ДТП/Съезд,ДТП/Опрокидывание,Не справился с управлением,Неисправность транспортного средства,Нарушение ПДД,ДТП/Падение пассажира,Возгорание транспортого средства
0,2014-01-07,0,0,0,0,0,0,0,0,0
1,2014-12-21,0,0,0,0,0,0,0,0,0
2,2015-01-01,0,0,0,0,0,0,0,0,0


In [52]:
# df_car_acidents, df_accidents
replacements = {
    "стол": "ДТП/Столкновение",
    "наез": "ДТП/Наезд",
    "опрок": "ДТП/Опрокидывание",
    "опрак": "ДТП/Опрокидывание",
    "упал": "ДТП/Опрокидывание",
    "опрк": "ДТП/Опрокидывание",
    "съе": "ДТП/Съезд",
    "кьюве": "ДТП/Съезд",
    "нарушен": "Нарушение ПДД",
    "неисправн": "Неисправность транспортного средства",
    "падени": "ДТП/Падение пассажира",
    "выпа": "ДТП/Падение пассажира",
    "возг": "Возгорание транспортого средства",
    "загор": "Возгорание транспортого средства",
    "кз": "Неисправность транспортного средства",
    "к/з": "Неисправность транспортного средства",
    "справ": "Не справился с управлением"

}

for ind, date in enumerate(y['Дата']):
    accidents_slice =  df_accidents[df_accidents['Дата'] == date]
    car_accidents_slice =  df_car_acidents[df_car_acidents['Дата'] == date]
    # print(accidents_slice['Тип'])
    # print(car_accidents_slice.columns)
    for event in accidents_slice['Тип']:
        if not isinstance(event, float):
            for key in replacements:
                if key in event.lower():
                    event_type = replacements[key]
                    y.at[ind, event_type] = 1
                    # y.iloc[ind][event_type] = 1
                    break
                
    for event in car_accidents_slice['Причина']:
        if not isinstance(event, float):
            for key in replacements:
                if key in event.lower():
                    event_type = replacements[key]
                    y.at[ind, event_type] = 1
                    # y.iloc[ind][event_type] = 1
                    break

y = y[[*y.columns[1:]]]

In [53]:
y.head()

,ДТП/Столкновение,ДТП/Наезд,ДТП/Съезд,ДТП/Опрокидывание,Не справился с управлением,Неисправность транспортного средства,Нарушение ПДД,ДТП/Падение пассажира,Возгорание транспортого средства
0,1,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0,0,0
2,1,1,0,0,0,0,0,0,0
3,1,1,0,0,0,1,0,0,0
4,1,0,0,0,0,0,0,0,0


train-test split

In [54]:
split_mark = int(0.8 * len(X))

X_train, X_test = X[:split_mark], X[split_mark:]
y_train, y_test = y[:split_mark], y[split_mark:]

X_train.to_csv('X_train.csv')
X_test.to_csv('X_test.csv')
y_train.to_csv('y_train.csv')
y_test.to_csv('y_test.csv')

### Train&evaluate Randomforest

In [55]:
# Load your dataset
# X, y = load_data()
# catboost.VERBOSE = -1

# # Split the data into training and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# model = CatBoostClassifier(iterations=500, learning_rate=0.1, depth=3, loss_function='Logloss')

# Choose a multi-label classifier (e.g., RandomForest)
classifier = MultiOutputClassifier(RandomForestClassifier())

X.to_csv("X_DTP.csv", index=False)
# Train the model
classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average=None)



print("Accuracy:", accuracy, )
print("Precision:", precision.mean())

Accuracy: 0.1875
Precision: 0.30092592592592593


/home/eresque/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Data for GKH

In [56]:
data = pd.read_csv('fortab')
features = data.drop('Тип', axis=1)  # Замените 'target_column' на имя вашего столбца с метками
features = features[features.columns[3:]]
mean_values = features.median()
features.fillna(mean_values, inplace=True)
features = features.fillna('', downcast='infer')
features = features.select_dtypes(exclude=['object'])

target = data['Тип']

/tmp/ipykernel_102810/815636013.py:4: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  mean_values = features.median()


In [57]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [58]:
classifier = TabPFNClassifier(device='cuda', N_ensemble_configurations=32)

Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters


In [59]:
classifier.fit(X_train, y_train, overwrite_warning=True)

TabPFNClassifier(N_ensemble_configurations=32)

In [60]:
y_eval, p_eval = classifier.predict(X_test, return_winning_probability=True)

In [61]:
print('Accuracy', accuracy_score(y_test, y_eval))

Accuracy 0.7972972972972973 Recall
